In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

COLOR_MODE = "grayscale"
CHANNELS = 1 if COLOR_MODE == "grayscale" else 3
IMAGE_HEIGHT = 150
IMAGE_WIDTH = 150

TRAINING_PATH = "./retina_dataset/Training_Set/Training_Set"
TRAINING_PATH_CSV = f"{TRAINING_PATH}/RFMiD_Training_Labels.csv"

TESTING_PATH = "./retina_dataset/Test_Set/Test_Set"
TESTING_PATH_CSV = f"{TESTING_PATH}/RFMiD_Testing_Labels.csv"

VALIDATION_PATH = "./retina_dataset/Evaluation_Set/Evaluation_Set"
VALIDATION_PATH_CSV = f"{VALIDATION_PATH}/RFMiD_Validation_Labels.csv"


train_df = pd.read_csv(TRAINING_PATH_CSV)
train_df = train_df[["ID","Disease_Risk"]]

test_df = pd.read_csv(TESTING_PATH_CSV)
test_df = test_df[["ID","Disease_Risk"]]

validation_df = pd.read_csv(VALIDATION_PATH_CSV)
validation_df = validation_df[["ID","Disease_Risk"]]


# print(train_df["Disease_Risk"].value_counts(normalize=True))
print(train_df)

for i in train_df:
    print(i)

        ID  Disease_Risk
0        1             1
1        2             1
2        3             1
3        4             1
4        5             1
...    ...           ...
1915  1916             1
1916  1917             1
1917  1918             0
1918  1919             0
1919  1920             0

[1920 rows x 2 columns]
ID
Disease_Risk


In [2]:
def append_ext(id):
        return f"{id}.png"

def convert_class(label):
        if label == 0:
            return "no"
        elif label == 1:
            return "yes"

train_df['name'] = train_df['ID'].apply(append_ext)
test_df['name'] = test_df['ID'].apply(append_ext)
validation_df['name'] = validation_df['ID'].apply(append_ext)

train_df['class'] = train_df['Disease_Risk']
test_df['class'] = test_df['Disease_Risk']
validation_df['class'] = validation_df['Disease_Risk']
# .apply(convert_class)

train_df = train_df.drop(["ID", "Disease_Risk"], axis=1)
test_df = test_df.drop(["ID", "Disease_Risk"], axis=1)
validation_df = validation_df.drop(["ID", "Disease_Risk"], axis=1)


In [3]:
display(train_df)
# display(test_df)
# display(validation_df)

,name,class
0,1.png,1
1,2.png,1
2,3.png,1
3,4.png,1
4,5.png,1
...,...,...
1915,1916.png,1
1916,1917.png,1
1917,1918.png,0
1918,1919.png,0


In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


def get_images_labels(image_folder_path, dataframe):
    # Load and preprocess images
	images = []
	labels = []

	for index, row in dataframe.iterrows():
		image_path = os.path.join(image_folder_path, row['name'])
		image = load_img(
			image_path,
			color_mode=COLOR_MODE,
			target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
			interpolation='nearest',
		)  # Assuming images are resized to 128x128 pixels
		image = img_to_array(image) / 255.0  # Rescale pixel values to [0, 1]
		images.append(image)
		labels.append(row['class'])

	# Convert lists to NumPy arrays
	images = np.array(images)
	labels = np.array(labels)

	return images, labels

train_ds_images, train_ds_labels = get_images_labels(f"{TRAINING_PATH}/Training", train_df)
test_ds_images, test_ds_labels = get_images_labels(f"{TESTING_PATH}/Test", test_df)
validation_ds_images, validation_ds_labels = get_images_labels(f"{VALIDATION_PATH}/Validation", validation_df)


In [5]:
# # Split data into training and testing sets
# x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
import numpy as np
import cv2

def random_flip(image):
    # Random horizontal and vertical flip
    if np.random.rand() < 0.5:  # 50% probability for each flip
        image = np.flip(image, axis=0)  # Vertical flip
    if np.random.rand() > 0.5:
        image = np.flip(image, axis=1)  # Horizontal flip
    return image

def random_rotation(image):
    # Random rotation within the range [-1, 1] degree
    angle = np.random.uniform(-1, 1)
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    return rotated_image

def random_translation(image, max_translation_x, max_translation_y):
    # Check if the image is grayscale
    if len(image.shape) == 2:
        # Reshape the grayscale image to add the third dimension
        image = np.expand_dims(image, axis=-1)
    # Random translation by up to max_translation_x and max_translation_y
    rows, cols, channels = image.shape
    tx = np.random.uniform(-max_translation_x, max_translation_x) * cols
    ty = np.random.uniform(-max_translation_y, max_translation_y) * rows
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, M, (cols, rows))
    return translated_image

def random_zoom(image, min_zoom, max_zoom):
    # Random zoom within the range [min_zoom, max_zoom]
    zoom_factor = np.random.uniform(min_zoom, max_zoom)
    if len(image.shape) == 2:  # Grayscale image
        image = np.expand_dims(image, axis=-1)
    rows, cols, channels = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 0, zoom_factor)
    zoomed_image = cv2.warpAffine(image, M, (cols, rows))
    return zoomed_image

def random_brightness(image, max_delta):
    # Random brightness adjustment within the range [-max_delta, max_delta]
    delta = np.random.uniform(-max_delta, max_delta)
    adjusted_image = np.clip(image + delta, 0, 255)
    return adjusted_image

def random_contrast(image, lower, upper):
    # Random contrast adjustment within the range [lower, upper]
    alpha = np.random.uniform(lower, upper)
    adjusted_image = np.clip(alpha * image, 0, 255)
    return adjusted_image

# # Example usage
# image = np.random.rand(128, 128, 3) * 255  # Example image of size 128x128x3 (RGB)

# # Apply random transformations
# image = random_flip(image)
# image = random_rotation(image)
# image = random_translation(image, max_translation_x=0.2, max_translation_y=0.2)
# image = random_zoom(image, min_zoom=0.8, max_zoom=1.2)
# image = random_brightness(image, max_delta=0.2)
# image = random_contrast(image, lower=0.8, upper=1.2)


original_images = train_ds_images
original_labels = train_ds_labels

def apply_augmentations(image):
    max_translation_x = 0.2
    max_translation_y=0.2
    min_zoom=0.8
    max_zoom=1.2
    max_brightness_delta=0.2
    min_contrast=0.8
    max_contrast=1.2
     
    augmented_image = image.copy()  # Make a copy of the original image
    # Apply random transformations to the image
    augmented_image = random_flip(augmented_image)
    augmented_image = random_rotation(augmented_image)
    augmented_image = random_translation(augmented_image, max_translation_x, max_translation_y)
    augmented_image = random_zoom(augmented_image, min_zoom, max_zoom)
    augmented_image = random_brightness(augmented_image, max_brightness_delta)
    augmented_image = random_contrast(augmented_image, min_contrast, max_contrast)
    return augmented_image

# Define the desired number of augmented images per original image
num_augmented_per_image = 5  # Adjust as needed
# Initialize lists to store augmented images and their corresponding labels
augmented_images = []
augmented_labels = []
# Apply augmentations to each original image
for _ in range(num_augmented_per_image):
    for i, image in enumerate(original_images):
        # Apply data augmentation techniques to the image
        augmented_image = apply_augmentations(image)
        augmented_images.append(augmented_image)
        # Repeat the corresponding label for the augmented images
        augmented_labels.append(original_labels[i])
        
# Convert the lists of augmented images and labels to NumPy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
# Verify the shapes of the augmented dataset and labels
print("Shape of augmented dataset:", augmented_images.shape)
print("Shape of augmented labels:", augmented_labels.shape)


Shape of augmented dataset: (9600, 150, 150)
Shape of augmented labels: (9600,)


In [7]:
import keras

# # Define the CNN model
# model = Sequential([
#     Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

model = keras.models.Sequential([
    # keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)),

    # preprocessing,

    # keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    keras.layers.Conv2D(32, 3, padding="same", activation="relu",input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)),
    keras.layers.MaxPooling2D(),
    # keras.layers.Dropout(0.2),

    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    # keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    # keras.layers.Dropout(0.2),

    keras.layers.Conv2D(256, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    # keras.layers.Dropout(0.2),

    # keras.layers.GlobalAveragePooling2D(),

    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1, activation="sigmoid"), #sigmoid is for binary classification
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.summary(
    expand_nested=True,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 128)      0

In [8]:
# print(len(train_ds_images), len(train_ds_labels))
# print(len(validation_ds_images), len(validation_ds_labels))
print(len(augmented_images), len(augmented_labels))
print(len(validation_ds_images), len(validation_ds_labels))
augmented_images.shape

9600 9600
640 640


(9600, 150, 150)

In [9]:
# # Train the model
EPOCHS = 100
PATIENCE = 25
BATCH_SIZE = 64
earlystop_loss = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

# Train the model
history = model.fit(
    x = augmented_images,
    y = augmented_labels,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(validation_ds_images, validation_ds_labels),
    callbacks=[earlystop_loss],
)

Epoch 1/100
150/150 [==============================] - 23s 114ms/step - loss: 0.5090 - binary_accuracy: 0.7906 - val_loss: 0.5080 - val_binary_accuracy: 0.7906
Epoch 2/100
150/150 [==============================] - 17s 110ms/step - loss: 0.4994 - binary_accuracy: 0.7911 - val_loss: 0.5349 - val_binary_accuracy: 0.7906
Epoch 3/100
150/150 [==============================] - 17s 111ms/step - loss: 0.4976 - binary_accuracy: 0.7911 - val_loss: 0.4987 - val_binary_accuracy: 0.7906
Epoch 4/100
150/150 [==============================] - 17s 111ms/step - loss: 0.4936 - binary_accuracy: 0.7911 - val_loss: 0.4994 - val_binary_accuracy: 0.7906
Epoch 5/100
150/150 [==============================] - 17s 111ms/step - loss: 0.4920 - binary_accuracy: 0.7911 - val_loss: 0.5223 - val_binary_accuracy: 0.7906
Epoch 6/100
150/150 [==============================] - 17s 110ms/step - loss: 0.4867 - binary_accuracy: 0.7911 - val_loss: 0.5119 - val_binary_accuracy: 0.7906
Epoch 7/100
150/150 [===================

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds_images, test_ds_labels)
print("Test Accuracy:", test_accuracy)

20/20 [==============================] - 1s 18ms/step - loss: 0.4083 - binary_accuracy: 0.8047
Test Accuracy: 0.8046875
